# Bonus: Secrets in Remote Runs

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()

In [ ]:
print("Workspace name: " + ws.name, 
      "Azure region: " + ws.location, 
      "Subscription id: " + ws.subscription_id, 
      "Resource group: " + ws.resource_group, sep = '\n')

Sometimes, you may have to pass a secret to a remote run, for example username and password to authenticate against external data source.

Azure ML SDK enables this use case through Key Vault associated with your workspace. The workflow for adding a secret is following.

On local computer:

 1. Read in a local secret, for example from environment variable or user input. To keep them secret, do not insert secret values into code as hard-coded strings.
 2. Obtain a reference to the keyvault
 3. Add the secret name-value pair in the key vault.
 
The secret is then available for remote runs as shown further below.

__Note__: The _azureml.core.keyvault.Keyvault_ is different from _azure.keyvault_ library. It is intended as simplified wrapper for setting, getting and listing user secrets in Workspace Key Vault.

In [ ]:
import os, uuid

local_secret = os.environ.get("LOCAL_SECRET", default = str(uuid.uuid4())) # Use random UUID as a substitute for real secret.
keyvault = ws.get_default_keyvault()
keyvault.set_secret(name="secret-name", value = local_secret)

The _set_secret_ method adds a new secret if one doesn't exist, or updates an existing one with new value.
You can list secret names you've added. This method doesn't return the values of the secrets.

In [ ]:
keyvault.list_secrets()

You can retrieve the value of the secret, and validate that it matches the original value. 

__Note__: This method returns the secret value. Take care not to write the the secret value to output.

In [ ]:
retrieved_secret = keyvault.get_secret(name="secret-name")
local_secret==retrieved_secret

In submitted runs on local and remote compute, you can use the get_secret method of Run instance to get the secret value from Key Vault. 

The method gives you a simple shortcut: the Run instance is aware of its Workspace and Keyvault, so it can directly obtain the secret without you having to instantiate the Workspace and Keyvault within remote run.

__Note__: This method returns the secret value. Take care not to write the secret to output.

For example, let's create a simple script _get_secret.py_ that gets the secret we set earlier. In an actual appication, you would use the secret, for example to access a database or other password-protected resource.

In [ ]:
%%writefile get_secret.py

from azureml.core import Run

run = Run.get_context()
secret_value = run.get_secret(name="secret-name")
print("Got secret value {} , but don't write it out!".format(len(secret_value) * "*"))

Then, submit the script as a regular script run, and find the obfuscated secret value in run output. You can use the same approach to other kinds of runs, such as Estimator ones.

In [ ]:
from azureml.core import Experiment, Run
from azureml.core.script_run_config import ScriptRunConfig

exp = Experiment(workspace = ws, name="try-secret")
src = ScriptRunConfig(source_directory=".", script="get_secret.py")

run = exp.submit(src)
run.wait_for_completion(show_output=True)

Furthermore, you can set and get multiple secrets using set_secrets and get_secrets methods.